In [1]:
import gpflow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("white")
sns.set_context("paper", font_scale=1.4)
import sys
sys.path.append('../')
from GPCollabPrefAct import real_occ_collab_gpmc

RuntimeError: module compiled against API version 0xb but this version of numpy is 0xa

RuntimeError: module compiled against API version 0xb but this version of numpy is 0xa

In [2]:
action_data_file = '../data/raw_data/main_actions_data.xlsx'

In [3]:
main_data = pd.read_excel(open(action_data_file, 'rb'), sheet_name = 0)

In [4]:
pref_rel_actions = main_data[main_data['Want more view'] == 0]
pref_rel_actions = pref_rel_actions[pref_rel_actions['Want more privacy'] == 0]

In [5]:
all_occ_wi_prev = pref_rel_actions['work plane illuminance before action']
all_occ_wi_cur = pref_rel_actions['work plane illuminance after action']

In [6]:
concat_all_occ_wi = np.hstack([all_occ_wi_cur, all_occ_wi_prev])
wi_mean = np.mean(concat_all_occ_wi) 
wi_std = np.std(concat_all_occ_wi)

In [7]:
print 'mean WI:'
print wi_mean
print 'std mean:'
print wi_std

mean WI:
924.908411215
std mean:
570.281070006


In [8]:
norm1 = (concat_all_occ_wi - 1200)/1200

In [9]:
occ_ids = np.unique(np.array(pref_rel_actions[['Occupant ID']]))

In [10]:
pref_rel_actions_data_list = []
pref_rel_actions_data_norm_list = []
for occ in occ_ids:
    wi_prev = np.array(pref_rel_actions[pref_rel_actions.iloc[:,0] == occ][['work plane illuminance before action']])
    wi_after = np.array(pref_rel_actions[pref_rel_actions.iloc[:,0] == occ][['work plane illuminance after action']])
    pref_rel_actions_mat = np.hstack([wi_prev, wi_after])
    pref_rel_actions_norm_mat = (pref_rel_actions_mat - wi_mean)/wi_std
    
    pref_rel_actions_data_list.append(pref_rel_actions_mat)
    pref_rel_actions_data_norm_list.append(pref_rel_actions_norm_mat)

In [11]:
pref_rel_actions_data_norm_list[2]

array([[-0.21902956, -0.78892398],
       [ 1.37649947, -1.08386626],
       [-1.27114234, -0.90535779],
       [ 0.4031198 , -0.55973875],
       [ 0.20847893, -0.94218174]])

In [12]:
X_unique = np.unique(concat_all_occ_wi)
X_grid = np.sort(X_unique)
X_grid_norm = (X_grid - wi_mean)/wi_std

In [115]:
def occ_prev_cur_action_ind(action_data_matrix, X_grid):
    """
    Inputs:
    actions_data_matrix : action data matrix (X_prev, X_current)
    Outputs:
    Concatenation of [X_cur, X_prev]
    Index of action data in X_grid
    """
    prev = action_data_matrix[:,0][:,None]
    cur = action_data_matrix[:,1][:,None]
    
    grid_sorted = np.argsort(X_grid)
    
    ypos_prev = np.searchsorted(X_grid[grid_sorted], prev)
    prev_ind = grid_sorted[ypos_prev]
    
    ypos_cur = np.searchsorted(X_grid[grid_sorted], cur)
    cur_ind = grid_sorted[ypos_cur]
    
    return prev_ind, cur_ind

def all_occ_prev_cur_act_ind(action_data_matrix_list, X_grid):
    """
    Inputs:
    action_data_matrix_list : list of all actions' data matrix (num_list = number of occupants)
    Outputs: 
    prev_ind_list : previous feature indices list
    cur_ind_list : current feature indices list
    """
    prev_ind_list = []
    cur_ind_list = []
    for i, mat in enumerate(action_data_matrix_list):
        prev_ind, cur_ind =  occ_prev_cur_action_ind(mat, X_grid)
        prev_ind_list.append(prev_ind[:,0])
        cur_ind_list.append(cur_ind[:,0])
    
    return prev_ind_list, cur_ind_list


def gather_concerned_utilities(utility_matrix, concerned_ind_mat):
    """
    Gather concerned utility function values for each occupant
    """
    
    splits = tf.split(concerned_ind_mat, num_or_size_splits=concerned_ind_mat.shape[1],axis = 1)
    res_list = []
    for i,s in enumerate(splits):
        #print i
        idx = tf.stack([tf.reshape(tf.range(utility_matrix.shape[0]), (-1,1)), tf.to_int32(s)], axis=-1)
        res = tf.gather_nd(utility_matrix, idx)
        res_flatten = tf.reshape(res, [-1])
        res_list.append(res_flatten)
    concerned_mat = tf.transpose(tf.stack(res_list)) 
    return concerned_mat

def select_specific_column_indices(ind_list):
    """
    Select specific column indices corresponding to specific row indices 
    Inputs:
    ind_list : list of features (previous or current) of all occupants
    Outputs:
    rel_tf_ind : relevant tensor for selecting specific columns from each rows.
    """
    total_ind_list = []
    for i in xrange(len(ind_list)):
        col = ind_list[i]
        row = np.repeat(i, col.shape[0])
        c_tf = tf.constant(col)
        r_tf = tf.constant(row)
        full_ind = tf.stack([r_tf, c_tf], axis=1)
        total_ind_list.append(full_ind)
    total_ind_tf = tf.concat(total_ind_list, 0)
    return total_ind_tf

In [116]:
prev_ind_list, cur_ind_list = all_occ_prev_cur_act_ind(pref_rel_actions_data_norm_list, X_grid_norm)

In [117]:
total_ind_tf = select_specific_column_indices(prev_ind_list)

In [118]:
c0 = prev_ind_list[0]
r0 = np.repeat(0, c0.shape[0])

In [119]:
c0_tf = tf.constant(c0)
r0_tf = tf.constant(r0)

# zip row indices with column indices
full_indices = tf.stack([r0_tf, c0_tf], axis=1)

In [123]:
U_rand = np.random.rand(len(prev_ind_list), X_grid_norm.shape[0])
U_rand_tf = tf.constant(U_rand)

In [124]:
U_rand

array([[ 0.38907599,  0.68418262,  0.85668479, ...,  0.65658868,
         0.04912722,  0.60110684],
       [ 0.27874058,  0.59975418,  0.0592662 , ...,  0.03501984,
         0.79282468,  0.42954314],
       [ 0.96580832,  0.03792772,  0.77610254, ...,  0.80188553,
         0.91143246,  0.01465425],
       ..., 
       [ 0.50941792,  0.13879544,  0.96517008, ...,  0.63241482,
         0.45946257,  0.46279086],
       [ 0.36254175,  0.65387704,  0.23587533, ...,  0.81532205,
         0.05498111,  0.12997859],
       [ 0.28755531,  0.64951261,  0.75335861, ...,  0.73757334,
         0.2616199 ,  0.48783031]])

In [125]:
# retrieve values by indices
S = tf.gather_nd(U_rand_tf, total_ind_tf)

In [126]:
with tf.Session() as sess:
    r = sess.run(S) 

In [127]:
U_rand.shape

(18, 203)

In [128]:
r

array([ 0.54192124,  0.82588405,  0.37432812,  0.4688539 ,  0.01673489,
        0.59764332,  0.69964865,  0.78689964,  0.24521743,  0.2502817 ,
        0.12121512,  0.84551317,  0.38570559,  0.34080544,  0.87221192,
        0.77610254,  0.06628387,  0.84006555,  0.50786459,  0.43677314,
        0.14447968,  0.80628028,  0.61558954,  0.38874921,  0.41171989,
        0.50882723,  0.36021586,  0.47147692,  0.16582305,  0.16153601,
        0.81431759,  0.38198041,  0.97304127,  0.35772618,  0.01051996,
        0.59041571,  0.23416839,  0.35772618,  0.92209844,  0.85900222,
        0.21670567,  0.22666694,  0.63846318,  0.97129116,  0.70427658,
        0.57249561,  0.72988686,  0.82558274,  0.61609535,  0.23266343,
        0.0357339 ,  0.18575667,  0.44267996,  0.70042197,  0.98075855,
        0.24885151,  0.04373139,  0.12954231,  0.48407535,  0.23391507,
        0.11117438,  0.25928916,  0.84881959,  0.30528219,  0.16813117,
        0.67978986,  0.88241171,  0.6770997 ,  0.65878871,  0.69

In [129]:
U_rand

array([[ 0.38907599,  0.68418262,  0.85668479, ...,  0.65658868,
         0.04912722,  0.60110684],
       [ 0.27874058,  0.59975418,  0.0592662 , ...,  0.03501984,
         0.79282468,  0.42954314],
       [ 0.96580832,  0.03792772,  0.77610254, ...,  0.80188553,
         0.91143246,  0.01465425],
       ..., 
       [ 0.50941792,  0.13879544,  0.96517008, ...,  0.63241482,
         0.45946257,  0.46279086],
       [ 0.36254175,  0.65387704,  0.23587533, ...,  0.81532205,
         0.05498111,  0.12997859],
       [ 0.28755531,  0.64951261,  0.75335861, ...,  0.73757334,
         0.2616199 ,  0.48783031]])